<a href="https://colab.research.google.com/github/snpushpi/6.804-Computational-Cognitive-Science/blob/master/Nested_Particle_Filter_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import math
import scipy.stats

In [ ]:
state_set = {'A','B','C','D'}
mu_dict = {'A':.2,'B':.4,'C':.6,'D':.8}
sigma = 0.1

In [ ]:
def weight_based_sampling(S): #[[state,weight]]
    states=[e[0] for e in S] 
    weights = [e[1] for e in S]
    state =  np.random.choice(states,p=weights) #states [[][]]
    weight = None
    for elt in S:
        if elt[0]==state:
            weight = elt[1]
    return state,weight

In [ ]:
def weight_calculate(S):
    weight_dict = {'A':0,'B':0,'C':0,'D':0}
    for state,weight in S:
        weight_dict[state]+=weight
    return weight_dict

In [ ]:
def most_probable_state_prediction(particle_filter_list):
    '''Each element in this list is a particle filter itself. This function returns teh most weighed state of all the particle 
    filters. Recall that particle filters themselves are weighed too, so we keep that in mind in the computation too
    '''
    #weight check 
    check_weight = 0
    for particle_filter, weight in particle_filter_list:
        check_weight+=weight  
    weight_dictionary = {'A':0,'B':0,'C':0,'D':0}
    for particle_filter,particle_filter_weight in particle_filter_list:
        for elt in weight_dictionary:
            weight_dictionary[elt]+= particle_filter.weight_dict[elt]*particle_filter_weight
    max_prob_state = max(weight_dictionary, key=weight_dictionary.get)
    return max_prob_state, weight_dictionary 

In [ ]:
class Particle_Filter():

    def __init__(self,particle_num,r):
        self.n = particle_num
        self.S = [[random.choice(tuple(state_set)),1/particle_num] for i in range(particle_num)]
        self.r = r
        self.prediction = None
        self.weight_dict = None

    def update(self,observation):
        '''self.S gets updated at each trial.'''
        S_new = [] 
        eta = 0 
        for i in range(self.n):
            state,weight = weight_based_sampling(self.S) #a particle 
            x1 = random.uniform(0,1) 
            if x1<self.r: #change state - (1-e**(-r*k)) =>
                new_set = state_set-{state}
                state= random.choice(tuple(new_set))
            new_weight = scipy.stats.norm(mu_dict[state],sigma).pdf(observation)
            eta+= new_weight
            S_new.append([state,new_weight])
        S_new = [[elt[0],elt[1]/eta] for elt in S_new]
        self.weight_dict = weight_calculate(S_new)
        self.prediction = max(self.weight_dict, key=self.weight_dict.get)
        self.S = S_new

In [ ]:
def nested_main(particle_filter_num, observation_list, human_prediction_list, particle_num, r):
    #main challenge - how to weight the particle filters themselves?
    #working on the base that the particle filters whoch are more consistent with the human rpediction should be weghed high
    #The way I do it is the following - 
    #After any trial, the weight of a particle filter is the probability with which it is generating the observant's prediction 
    particle_filter_list = [[Particle_Filter(particle_num,r),1/particle_filter_num] for i in range(particle_filter_num)]
    model_prediction = []
    for i in range(len(observation_list)):
        eta = 0
        H_new = []
        particle_filter_keeper = []
        #model_prediction.append(most_probable_state_prediction(particle_filter_list))
        for particle_filter,weight in particle_filter_list:
            particle_filter.update(observation_list[i])
            particle_filter_keeper.append([particle_filter,weight])
            weight = particle_filter.weight_dict[human_prediction_list[i]] 
            H_new.append([particle_filter,weight])
            eta+=weight
        most_prob_state, weight_dict = most_probable_state_prediction(particle_filter_keeper)
        model_prediction.append([most_prob_state,weight_dict[human_prediction_list[i]]])
        H_new = [[elt[0],elt[1]/eta] for elt in H_new]
        #At this point, we will get the model prediction, which we are actually getting from the weighed particle filters,
        #In other words, each particle filter is consistent with human prediction and the more it is consistent, the higherweight it gets
        #Now to get the model prediction as a whole, or the prediction from researcher filter would be the weighed prediction from all partcile filters
        # and the state getting highest weight this way is the researcher's prediction. This role is being done by the most_probable_state_prediction
        #H_New = [[particle_filter, weight]]
        particle_filter_index_list = [[i,H_new[i][1]] for i in range(particle_filter_num)]
        new_list = []
        weight_sum = 0
        for j in range(particle_filter_num):
            index, weight = weight_based_sampling(particle_filter_index_list)
            weight_sum+= weight
            new_list.append([H_new[index][0],weight])
        particle_filter_list =  [[elt[0],elt[1]/weight_sum] for elt in new_list]
    return model_prediction

In [ ]:
def accuracy_and_similarity(human_inference_list, particle_number, particle_filter_num, observation_list, actual_list, r):
    '''This function returns both the task accuracy and human similarity measure of the normal
    particle filter model'''
    model_inference_list = nested_main(particle_filter_num, observation_list, human_inference_list, particle_number, r)
    human_similarity_measure = 0
    task_counter = 0
    for i in range(len(observation_list)):
        human_similarity_measure+=model_inference_list[i][1]
        if actual_list[i]==model_inference_list[i][0]:
            task_counter+=1
    return human_similarity_measure/len(observation_list),task_counter/len(observation_list)


In [ ]:
def data_generate(alpha,T):
    '''Goal is generate T observations with state changing probability alpha.
    Step 1: Set Z_0 as a random sample from state list.
    Step 2: Repeat the following for T trials -
          i) Z_i = Z_i-1
          ii)Sample x randomly from [0,1]
          iii) If x<alpha replace Z_i with random sample {A,B,C,D}/Z_i-1
          iv)Sample stimulus y_i form a normal distribution with std sigma and mu_zi
    '''
    observation_list = []
    Z = [None]*(T+1)
    Z[0] = random.choice(tuple(state_set))
    for i in range(1,T+1):
        Z[i]=Z[i-1]
        x = random.uniform(0,1)
        if x<alpha:
            new_set = state_set-{Z[i-1]}
            Z[i]= random.choice(tuple(new_set))
        observation_list.append(random.gauss(mu_dict[Z[i]],sigma))        
    return observation_list,Z[1:]

In [ ]:
observation_list,actual_list = data_generate(0.16,20)


In [ ]:
observation_list = [0.2876251534356482,
 0.3093255562034109,
 0.48407616095419964,
 0.4837637156934066,
 0.3429542790801123,
 0.8145469658341187,
 0.5052333561135628,
 0.566893813089657,
 0.5190551990406111,
 0.6756708844580039,
 0.5941568333778058,
 0.6689426081101612,
 0.08007089954529747,
 0.24591174989381062,
 0.13612431798914626,
 0.16855115278807364,
 0.02849659237553981,
 0.2997500771551358,
 0.3786856816409396,
 0.17848722367921957]

In [ ]:
actual_list = ['B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'D',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A']

In [ ]:
human_inference_list = ['B','B','C','C','C','C','C','D','D','D','D','D','A','A',
 'A','A','A','A','B','B']
particle_number = 5
r = 0.2
particle_filter_num = 200
accuracy_and_similarity(human_inference_list, particle_number, particle_filter_num, observation_list, actual_list, r)

(0.45047769340560145, 0.65)